In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

In [7]:
spark = SparkSession.builder.master("local[*]").appName("Demo").getOrCreate()
spark

In [28]:
empDf = spark.read.option("header",True).option("infrechema",True).csv("../Spark-main/employees.csv")
empDf.show(1)

AnalysisException: Path does not exist: file:/home/pradeep.k@zucisystems.com/workspace_working_Repo/Spark-main/employees.csv

In [9]:
deptDf = spark.read.option("header",True).option("infrechema",True).csv("Spark-main/departments.csv")
deptDf.show(1)

+-------------+---------------+----------+-----------+
|DEPARTMENT_ID|DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+---------------+----------+-----------+
|           10| Administration|       200|       1700|
+-------------+---------------+----------+-----------+
only showing top 1 row



In [10]:
from pyspark.sql.functions import *

In [11]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 104857600)
#The Value is in Bytes

In [12]:
empDf.join(broadcast(deptDf), empDf.DEPARTMENT_ID == deptDf.DEPARTMENT_ID, "inner").select(empDf.EMPLOYEE_ID, empDf.DEPARTMENT_ID, deptDf.DEPARTMENT_NAME).show(100)

+-----------+-------------+----------------+
|EMPLOYEE_ID|DEPARTMENT_ID| DEPARTMENT_NAME|
+-----------+-------------+----------------+
|        198|           50|        Shipping|
|        199|           50|        Shipping|
|        200|           10|  Administration|
|        201|           20|       Marketing|
|        202|           20|       Marketing|
|        203|           40| Human Resources|
|        204|           70|Public Relations|
|        205|          110|      Accounting|
|        206|          110|      Accounting|
|        100|           90|       Executive|
|        101|           90|       Executive|
|        102|           90|       Executive|
|        103|           60|              IT|
|        104|           60|              IT|
|        105|           60|              IT|
|        106|           60|              IT|
|        107|           60|              IT|
|        108|          100|         Finance|
|        109|          100|         Finance|
|        1